In [ ]:
!pip install icrawler
import os
from icrawler.builtin import GoogleImageCrawler

ingredients = ['onion', 'carrot', 'potato', 'pork', 'eggplant', 'green_pepper']

keywords = ['玉ねぎ 袋', 'にんじん 袋', 'きゅうり 袋', 'じゃがいも 袋', 'ピーマン 袋']
dir_names = ['onion_bag', 'carrot_bag', 'cucumber_bag', 'potato_bag', 'green_pepper_bag']

# ダウンロードディレクトリを作成
for dir_name in dir_names:
    os.makedirs('./image/' + dir_name, exist_ok=True)

for i in range(len(keywords)):
    google_crawler = GoogleImageCrawler(
        storage={'root_dir': './image/' + dir_names[i]},
        downloader_threads=4,  # ダウンロードスレッドの数
        parser_threads=4,     # パーススレッドの数
    )
    google_crawler.crawl(keyword=keywords[i], max_num=100)  # ここで画像の数を指定します


In [ ]:
!pip install icrawler
import os
from icrawler.builtin import BingImageCrawler

keywords = ['玉ねぎ 袋', 'にんじん 袋', 'きゅうり 袋', 'じゃがいも 袋', 'ピーマン 袋']
dir_names = ['onion_bag', 'carrot_bag', 'cucumber_bag', 'potato_bag', 'green_pepper_bag']

# ダウンロードディレクトリを作成
for dir_name in dir_names:
    os.makedirs('./image_bing/' + dir_name, exist_ok=True)

for i in range(len(keywords)):
    bing_crawler = BingImageCrawler(
        storage={'root_dir': './image_bing/' + dir_names[i]},
        downloader_threads=4,  # ダウンロードスレッドの数
        parser_threads=4,     # パーススレッドの数
    )
    bing_crawler.crawl(keyword=keywords[i], max_num=100)  # ここで画像の数を指定します


泥臭いけど手作業で画像を厳選する. 厳選した画像は'./selected_images'に格納する. 

In [ ]:
import os

source_dir = './selected_images'  # 画像の元のディレクトリ

selected_dir = ['carrot', 'cucumber', 'green_pepper', 'onion', 'potato']

# 各ディレクトリ内の画像ファイルの名前を番号で振りなおす
for dir_name in selected_dir:
    source_subdir = os.path.join(source_dir, dir_name)
    
    # サブディレクトリ内の画像ファイル一覧を取得
    image_files = [f for f in os.listdir(source_subdir) if os.path.isfile(os.path.join(source_subdir, f))]
    
    # 画像ファイルに番号を振りなおして名前を変更
    for i, filename in enumerate(image_files, start=1):
        file_ext = os.path.splitext(filename)[1]  # ファイルの拡張子を取得
        new_filename = f"{i:04d}{file_ext}"  # 4桁の番号で新しい名前を作成
        source_file_path = os.path.join(source_subdir, filename)
        new_file_path = os.path.join(source_subdir, new_filename)
        
        # ファイル名を変更
        os.rename(source_file_path, new_file_path)
        print(f"Rename: {filename} -> {new_filename}")

print("ファイル名の変更が完了しました。")

In [ ]:
!pip install opencv-python
import os
import cv2

# ディレクトリのパス
source_dir = './selected_images'  # 画像の元のディレクトリ
output_dir = './squared_images'  # 正方形に整形した画像の保存先ディレクトリ

selected_dir = ['carrot', 'cucumber', 'green_pepper', 'onion', 'potato']

# 保存先ディレクトリが存在しない場合は作成
os.makedirs(output_dir, exist_ok=True)

# 各ディレクトリ内の画像を正方形に整形して保存
for dir_name in selected_dir:
    source_subdir = os.path.join(source_dir, dir_name)
    output_subdir = os.path.join(output_dir, dir_name)
    os.makedirs(output_subdir, exist_ok=True)

    for filename in os.listdir(source_subdir):
        img_path = os.path.join(source_subdir, filename)
        output_path = os.path.join(output_subdir, filename)

        # 画像を読み込む
        img = cv2.imread(img_path)

        # 画像を正方形に整形
        size = max(img.shape[:2])
        square_img = cv2.resize(img, (size, size))

        # 画像を保存
        cv2.imwrite(output_path, square_img)

print("正方形に整形して保存が完了しました。")

data augmentationはGoogle Colabで実行した. ローカルだと何故か動かなかった. ローカルでsquared_imagesディレクトリをzipに圧縮した後, 以下のコードで一旦Google Colabに移植する. 

In [ ]:
from google.colab import files
uploaded_folder = files.upload()

In [ ]:
import zipfile
import io

# アップロードしたzipファイルの名前を指定
zip_file_name = "squared_images.zip"

# zipファイルを解凍
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("/content/")  # 解凍先のディレクトリを指定

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# 画像の格納ディレクトリと保存先ディレクトリを指定
image_dir = './squared_images/'
save_dir = './augmented_images/'

# データ拡張の設定
datagen = ImageDataGenerator(
    rotation_range=40,     # 画像を最大で40度回転
    width_shift_range=0.2, # 画像を水平にランダムにシフト
    height_shift_range=0.2,# 画像を垂直にランダムにシフト
    shear_range=0.2,       # シアー変換を適用
    zoom_range=0.2,        # ランダムにズーム
    horizontal_flip=True,  # 画像を水平に反転
    fill_mode='nearest'    # 境界周りのピクセルを補完
)

# ディレクトリ内の各キーワードの画像にデータ拡張を適用
for keyword in dir_names:
    image_subdir = os.path.join(image_dir, keyword)
    save_subdir = os.path.join(save_dir, keyword)
    os.makedirs(save_subdir, exist_ok=True)

    for filename in os.listdir(image_subdir):
        img_path = os.path.join(image_subdir, filename)
        img = load_img(img_path)
        img_array = img_to_array(img)
        img_array = img_array.reshape((1,) + img_array.shape)

        # データ拡張を適用して新しい画像を生成
        i = 0
        for batch in datagen.flow(img_array, save_to_dir=save_subdir, save_prefix='aug', save_format='jpeg'):
            i += 1
            if i >= 5:  # 各元画像から5枚のデータ拡張画像を生成
                break

In [8]:
import os
import cv2

# データ拡張後の画像が保存されたディレクトリと、圧縮後の画像を保存するディレクトリ
augmented_image_dir = './augmented_images/'

selected_dir = ['carrot', 'cucumber', 'green_pepper', 'onion', 'potato']

# 圧縮後の画像のサイズ
target_size = (64, 64)

# 圧縮後の画像を保存するディレクトリを作成
compressed_image_dir = './64_compressed_images/'
os.makedirs(compressed_image_dir, exist_ok=True)

# ディレクトリ内の各キーワードの画像を読み込んで圧縮
for keyword in selected_dir:
    augmented_subdir = os.path.join(augmented_image_dir, keyword)
    compressed_subdir = os.path.join(compressed_image_dir, keyword)
    os.makedirs(compressed_subdir, exist_ok=True)

    for filename in os.listdir(augmented_subdir):
        img_path = os.path.join(augmented_subdir, filename)
        img = cv2.imread(img_path)

        # 画像を指定のサイズにリサイズ
        compressed_img = cv2.resize(img, target_size)

        # 圧縮後の画像を保存
        compressed_path = os.path.join(compressed_subdir, filename)
        cv2.imwrite(compressed_path, compressed_img)

print("画像の圧縮が完了しました。")


画像の圧縮が完了しました。


In [10]:
import os
import random
import shutil
import csv

selected_dir = ['carrot', 'cucumber', 'green_pepper', 'onion', 'potato']

# ディレクトリのパス
source_dir = '/Users/hiro/Desktop/Sony_hackathon/main/64_compressed_images'
train_dir = '/Users/hiro/Desktop/Sony_hackathon/main/64_train/'
test_dir = '/Users/hiro/Desktop/Sony_hackathon/main/64_test/'
train_csv_filename = '/Users/hiro/Desktop/Sony_hackathon/main/64_train_labels.csv'  # トレーニング用CSVファイルの保存先
test_csv_filename = '/Users/hiro/Desktop/Sony_hackathon/main/64_test_labels.csv'    # テスト用CSVファイルの保存先

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# トレーニングセットとテストセットの比率
train_ratio = 0.8  # 80%をトレーニングセットとする

# トレーニング用とテスト用のCSVファイルのヘッダを設定
with open(train_csv_filename, mode='w', newline='') as train_csv_file:
    train_csv_writer = csv.writer(train_csv_file)
    train_csv_writer.writerow(['x', 'y'])  # ヘッダ行

with open(test_csv_filename, mode='w', newline='') as test_csv_file:
    test_csv_writer = csv.writer(test_csv_file)
    test_csv_writer.writerow(['x', 'y'])  # ヘッダ行

# 各ディレクトリ内の画像ファイルを読み込み、ランダムにシャッフル
for keyword_idx, keyword in enumerate(selected_dir):
    keyword_source_dir = os.path.join(source_dir, keyword)

    image_files = os.listdir(keyword_source_dir)
    random.shuffle(image_files)

    num_train = int(len(image_files) * train_ratio)

    for i, image_file in enumerate(image_files):
        src_path = os.path.join(keyword_source_dir, image_file)
        if i < num_train:
            dst_dir = train_dir
            csv_filename = train_csv_filename
        else:
            dst_dir = test_dir
            csv_filename = test_csv_filename
        dst_path = os.path.join(dst_dir, image_file)

        # 画像ファイルをコピー
        shutil.copy(src_path, dst_path)

        # CSVファイルにパスとラベルを追加
        with open(csv_filename, mode='a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([dst_path, keyword_idx])

print("画像の振り分けとCSVファイルの作成が完了しました。")


画像の振り分けとCSVファイルの作成が完了しました。


ここでは, NNCにアップロードするためにcsvファイルも一緒に生成してる. 